In [63]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\KK\AppData\Local\Temp\ipykernel_24588\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [64]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [65]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [66]:
# Your code
print(data.head())

from sklearn.model_selection import train_test_split

# Assuming 'data' contains the features and 'label_column' is the target column
X = data.drop(columns=['label'])  # remove label from the input ds to form training data
y = data['label']  # Label data moved to y

# Split the dataset into training and test sets, using 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the new sets
print(f"Training set size: {X_train.shape}, Test set size: {X_test.shape}")


                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0
3  Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                fyi      0
Training set size: (800, 1), Test set size: (200, 1)


## Data Preprocessing

In [67]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [68]:
# Your code

import re

def clean_html_with_regex(raw_html):
    # Step 1: Remove inline JavaScript/CSS (scripts and styles)
    cleaned = re.sub(r'<(script|style).*?>.*?</\1>', '', raw_html, flags=re.DOTALL)
    
    # Step 2: Remove HTML comments
    cleaned = re.sub(r'<!--.*?-->', '', cleaned, flags=re.DOTALL)
    
    # Step 3: Remove remaining HTML tags
    cleaned = re.sub(r'<.*?>', '', cleaned)

    return cleaned

# Apply the cleaning function to X_train and X_test, assuming 'text_column' contains the HTML data
X_train['text'] = X_train['text'].apply(clean_html_with_regex)
X_test['text'] = X_test['text'].apply(clean_html_with_regex)
print(X_train.head())

                                                  text
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535  I have not been able to reach oscar this am. W...
695  ; Huma Abedin B6I'm checking with Pat on the 5...
557  I can have it announced here on Monday - can't...
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [69]:
# Your code

import re

def clean_text(text):
    # Step 1: Remove special characters (except spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Step 2: Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Step 3: Remove all single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Step 4: Remove single characters from the start (e.g., 'a word' -> ' word')
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    
    # Step 5: Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Step 6: Remove prefixed 'b' (common issue from byte strings)
    text = re.sub(r'\bb\b', '', text)
    
    # Step 7: Convert text to lowercase
    text = text.lower()
    
    return text.strip()  # Step 8: Strip leading/trailing spaces

# Apply the function to your X_train and X_test cleaned_text columns
X_train['text'] = X_train['text'].apply(clean_text)
X_test['text'] = X_test['text'].apply(clean_text)
print(X_train.head())

                                                  text
29   regards mr nelson smithkindly reply me on my p...
535  have not been able to reach oscar this am we a...
695  huma abedin bim checking with pat on the will ...
557    can have it announced here on monday cant today
836  bank of africaagence san pedro bp san pedro co...


## Now let's work on removing stopwords
Remove the stopwords.

In [70]:
# Your code

# NLTK 
import nltk
from nltk.corpus import stopwords

# Download stopwords
nltk.download('stopwords')

# Get the English stopwords list
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    # Split the text into words and remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply the function to your already cleaned text in X_train and X_test
X_train['text_no_stopwords'] = X_train['text'].apply(remove_stopwords)
X_test['text_no_stopwords'] = X_test['text'].apply(remove_stopwords)

# Check the result
print(X_train[['text', 'text_no_stopwords']].head())

                                                  text  \
29   regards mr nelson smithkindly reply me on my p...   
535  have not been able to reach oscar this am we a...   
695  huma abedin bim checking with pat on the will ...   
557    can have it announced here on monday cant today   
836  bank of africaagence san pedro bp san pedro co...   

                                     text_no_stopwords  
29   regards mr nelson smithkindly reply private em...  
535         able reach oscar supposed send pdb receive  
695  huma abedin bim checking pat work jack jake re...  
557                        announced monday cant today  
836  bank africaagence san pedro bp san pedro cote ...  


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [71]:
# Your code
#import ntlk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

# Download the WordNet lemmatizer data:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to map NLTK POS tags to WordNet POS tags
def get_wordnet_pos(word):
    """Map NLTK POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0]  # Get the first character of the POS tag
    tag_dict = {"J": wordnet.ADJ,   # Adjective
                "N": wordnet.NOUN,  # Noun
                "V": wordnet.VERB,  # Verb
                "R": wordnet.ADV}   # Adverb
    return tag_dict.get(tag, wordnet.NOUN)  # Default to NOUN if no match
    
# Function to lemmatize text with POS tagging
def lemmatize_text(text):
    # Lemmatize each word in the text using the POS tags
    lemmatized_text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return lemmatized_text

# Apply the function to your already cleaned text without stopwords
X_train['cleaned_text_lemmatized'] = X_train['text_no_stopwords'].apply(lemmatize_text)
X_test['cleaned_text_lemmatized'] = X_test['text_no_stopwords'].apply(lemmatize_text)

# Check the result
print(X_train[['text_no_stopwords', 'cleaned_text_lemmatized']].head())

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                     text_no_stopwords  \
29   regards mr nelson smithkindly reply private em...   
535         able reach oscar supposed send pdb receive   
695  huma abedin bim checking pat work jack jake re...   
557                        announced monday cant today   
836  bank africaagence san pedro bp san pedro cote ...   

                               cleaned_text_lemmatized  
29   regard mr nelson smithkindly reply private ema...  
535         able reach oscar supposed send pdb receive  
695  huma abedin bim checking pat work jack jake re...  
557                        announced monday cant today  
836  bank africaagence san pedro bp san pedro cote ...  


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

# Step-by-Step Approach:
1. Separate spam and ham messages.
2. Count word frequencies for both spam and ham.
3. Get the top 10 words for each category.

In [74]:
# from IRONHACK AI
""" from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')

ham_messages = X_train[y_train == 0]['cleaned_text_lemmatized'].dropna()
spam_messages = X_train[y_train == 1]['cleaned_text_lemmatized'].dropna()

ham_bow = vectorizer.fit_transform(ham_messages)
spam_bow = vectorizer.fit_transform(spam_messages)

# Convert to DataFrame for word counts
ham_word_counts = pd.DataFrame(ham_bow.toarray(), columns=vectorizer.get_feature_names_out()).sum().sort_values(ascending=False)
spam_word_counts = pd.DataFrame(spam_bow.toarray(), columns=vectorizer.get_feature_names_out()).sum().sort_values(ascending=False)

top_ham_words = ham_word_counts.head(10)
top_spam_words = spam_word_counts.head(10)

print("Top 10 words in ham (Not Spam) messages:\n", top_ham_words)
print("Top 10 words in spam messages:\n", top_spam_words) """

' from sklearn.feature_extraction.text import CountVectorizer\n\nvectorizer = CountVectorizer(stop_words=\'english\')\n\nham_messages = X_train[y_train == 0][\'cleaned_text_lemmatized\'].dropna()\nspam_messages = X_train[y_train == 1][\'cleaned_text_lemmatized\'].dropna()\n\nham_bow = vectorizer.fit_transform(ham_messages)\nspam_bow = vectorizer.fit_transform(spam_messages)\n\n# Convert to DataFrame for word counts\nham_word_counts = pd.DataFrame(ham_bow.toarray(), columns=vectorizer.get_feature_names_out()).sum().sort_values(ascending=False)\nspam_word_counts = pd.DataFrame(spam_bow.toarray(), columns=vectorizer.get_feature_names_out()).sum().sort_values(ascending=False)\n\ntop_ham_words = ham_word_counts.head(10)\ntop_spam_words = spam_word_counts.head(10)\n\nprint("Top 10 words in ham (Not Spam) messages:\n", top_ham_words)\nprint("Top 10 words in spam messages:\n", top_spam_words) '

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Remove NaN values from 'cleaned_text_lemmatized' column in X_train
X_train['cleaned_text_lemmatized'] = X_train['cleaned_text_lemmatized'].replace('', float('NaN')).dropna()

# Separate ham (Not Spam: label = 0) and spam (Spam: label = 1) messages using y_train
ham_messages = X_train[y_train == 0]['cleaned_text_lemmatized']  # Not Spam
spam_messages = X_train[y_train == 1]['cleaned_text_lemmatized']  # Spam

# Ensure ham and spam messages are non-empty after cleaning
ham_messages = ham_messages.dropna().str.strip()  # Drop NaN and strip whitespaces
spam_messages = spam_messages.dropna().str.strip()  # Drop NaN and strip whitespaces

# Filter out empty strings (just in case there are messages that are empty after cleaning)
ham_messages = ham_messages[ham_messages != '']
spam_messages = spam_messages[spam_messages != '']

# Check if there are any remaining valid ham messages
if ham_messages.empty:
    print("No valid ham (Not Spam) messages after cleaning.")
else:
    # Initialize the CountVectorizer (Bag of Words)
    vectorizer = CountVectorizer(stop_words='english')
    
    # Fit the vectorizer to ham messages
    ham_bow = vectorizer.fit_transform(ham_messages)
    ham_words = vectorizer.get_feature_names_out()
    
    # Convert the results to DataFrame and get top words
    ham_word_counts = pd.DataFrame(ham_bow.toarray(), columns=ham_words).sum().sort_values(ascending=False)
    top_ham_words = ham_word_counts.head(10)
    print("Top 10 words in ham (Not Spam) messages:\n", top_ham_words)

# Check if there are any remaining valid spam messages
if spam_messages.empty:
    print("No valid spam messages after cleaning.")
else:
    # Fit the vectorizer to spam messages
    spam_bow = vectorizer.fit_transform(spam_messages)
    spam_words = vectorizer.get_feature_names_out()
    
    # Convert the results to DataFrame and get top words
    spam_word_counts = pd.DataFrame(spam_bow.toarray(), columns=spam_words).sum().sort_values(ascending=False)
    top_spam_words = spam_word_counts.head(10)
    print("Top 10 words in spam messages:\n", top_spam_words)


Top 10 words in ham (Not Spam) messages:
 state        92
pm           89
president    84
percent      76
secretary    71
mr           70
time         70
work         63
said         59
american     58
dtype: int64
Top 10 words in spam messages:
 money          795
account        662
bank           606
fund           565
business       393
transaction    347
country        337
transfer       326
company        318
million        315
dtype: int64


## Extra features

In [52]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

X_train['money_mark'] = X_train['cleaned_text_lemmatized'].str.contains(money_simbol_list)*1
X_train['suspicious_words'] = X_train['cleaned_text_lemmatized'].str.contains(suspicious_words)*1

X_test['money_mark'] = X_test['cleaned_text_lemmatized'].str.contains(money_simbol_list)*1
X_test['suspicious_words'] = X_test['cleaned_text_lemmatized'].str.contains(suspicious_words)*1

# Fill NaN values with an empty string before calculating text length
X_train['cleaned_text_lemmatized'] = X_train['cleaned_text_lemmatized'].fillna('')
X_train['text_len'] = X_train['cleaned_text_lemmatized'].apply(lambda x: len(x)) 

# Fill NaN values with an empty string before calculating text length
X_test['cleaned_text_lemmatized'] = X_test['cleaned_text_lemmatized'].fillna('')
X_test['text_len'] = X_test['cleaned_text_lemmatized'].apply(lambda x: len(x)) 

X_train.head()

,text,text_no_stopwords,cleaned_text_lemmatized,money_mark,suspicious_words,text_len
29,regards mr nelson smithkindly reply me on my p...,regards mr nelson smithkindly reply private em...,regard mr nelson smithkindly reply private ema...,1,0,75
535,have not been able to reach oscar this am we a...,able reach oscar supposed send pdb receive,able reach oscar supposed send pdb receive,1,0,42
695,huma abedin bim checking with pat on the will ...,huma abedin bim checking pat work jack jake re...,huma abedin bim checking pat work jack jake re...,1,0,79
557,can have it announced here on monday cant today,announced monday cant today,announced monday cant today,1,0,27
836,bank of africaagence san pedro bp san pedro co...,bank africaagence san pedro bp san pedro cote ...,bank africaagence san pedro bp san pedro cote ...,1,1,1067


## How would work the Bag of Words with Count Vectorizer concept?

In [54]:
# Your code

# Create a Bag of Words model using CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
vectorizer = CountVectorizer(stop_words='english')

# Fit the vectorizer to the training data (cleaned and lemmatized text) & create Bag of Words(bow)
X_train_bow = vectorizer.fit_transform(X_train['cleaned_text_lemmatized'])

# Apply the vectorizer to the test data & create Bag of Words(bow)
X_test_bow = vectorizer.transform(X_test['cleaned_text_lemmatized'])

# View the shape of the transformed data
print(X_train_bow.shape)  # (number_of_documents, size_of_vocabulary)

# To view the feature names (words in the vocabulary)
print(vectorizer.get_feature_names_out())

# Convert to dense array (for illustrative purposes only)
X_train_bow_array = X_train_bow.toarray()

# Example: Get the first transformed document
print(X_train_bow_array[0])


(800, 16740)
['aac' 'aaclocated' 'aae' ... 'zumadirector' 'zumae' 'zurich']
[0 0 0 ... 0 0 0]


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [55]:
# Your code

from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit the vectorizer on the training data and transform both train and test sets
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['cleaned_text_lemmatized'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['cleaned_text_lemmatized'])

# Print the shape of the vectorized dataset
print(f"Shape of TF-IDF vectorized training data: {X_train_tfidf.shape}")
print(f"Shape of TF-IDF vectorized test data: {X_test_tfidf.shape}")

# Optional: To view the feature names (the words that make up the vocabulary)
print(tfidf_vectorizer.get_feature_names_out())


Shape of TF-IDF vectorized training data: (800, 16740)
Shape of TF-IDF vectorized test data: (200, 16740)
['aac' 'aaclocated' 'aae' ... 'zumadirector' 'zumae' 'zurich']


## And the Train a Classifier?

In [59]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Initialize the Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()

# Train the classifier on the TF-IDF vectorized training data
nb_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred = nb_classifier.predict(X_test_tfidf)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print the classification report
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 94.50%
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.92      0.95       125
           1       0.88      0.99      0.93        75

    accuracy                           0.94       200
   macro avg       0.94      0.95      0.94       200
weighted avg       0.95      0.94      0.95       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [60]:
# Your code
# Combination of N-grams & TF-IDF:

# TF-IDF Vectorizer with N-grams
tfidf_ngram_vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
X_train_tfidf_ngram = tfidf_ngram_vectorizer.fit_transform(X_train['cleaned_text_lemmatized'])
X_test_tfidf_ngram = tfidf_ngram_vectorizer.transform(X_test['cleaned_text_lemmatized'])

# Train the MultinomialNB Classifier
classifier = MultinomialNB()
classifier.fit(X_train_tfidf_ngram, y_train)

# Predict and Evaluate
y_pred_tfidf_ngram = classifier.predict(X_test_tfidf_ngram)
print(f"Accuracy (TF-IDF + N-grams): {accuracy_score(y_test, y_pred_tfidf_ngram) * 100:.2f}%")
print("Classification Report (TF-IDF + N-grams):\n", classification_report(y_test, y_pred_tfidf_ngram))


Accuracy (TF-IDF + N-grams): 94.50%
Classification Report (TF-IDF + N-grams):
               precision    recall  f1-score   support

           0       0.99      0.92      0.95       125
           1       0.88      0.99      0.93        75

    accuracy                           0.94       200
   macro avg       0.94      0.95      0.94       200
weighted avg       0.95      0.94      0.95       200

